# 基于RNN的数字识别

### 载入MNIST数据

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('../../proj2/第二天/练习/MNIST_data', one_hot=True)

/Users/liuyanfang/anaconda3/envs/tensorflow1.4/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Extracting ../../proj2/第二天/练习/MNIST_data/train-images-idx3-ubyte.gz
Extracting ../../proj2/第二天/练习/MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../../proj2/第二天/练习/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../../proj2/第二天/练习/MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
sess = tf.InteractiveSession()

In [3]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

In [4]:
def lstm_cell(lstm_size):
  return tf.contrib.rnn.BasicLSTMCell(lstm_size)

在这一部分的循环神经网络中，我们选择使用LSTM结构，这种结构能够有效的防止梯度消失的问题。以上函数能够根据指定的大小返回一个LSTM单元

In [5]:
stacked_lstm = tf.contrib.rnn.MultiRNNCell(
    [lstm_cell(28*28) for _ in range(3)]) # 参数是： cells, state_is_tuple=True

### 通过使用tf.contrib.rnn.MultiRNNCell，我们可以将多个LSTM连接起来形成多层RNN网络。
### Args:
      cells: list of RNNCells that will be composed in this order.
      state_is_tuple: If True, accepted and returned states are n-tuples, where
        `n = len(cells)`.  If False, the states are all
        concatenated along the column axis.  This latter behavior will soon be
        deprecated.

In [6]:
state = stacked_lstm.zero_state(batch_size=tf.shape(x)[0], dtype=tf.float32)

定义RNN的初始状态为零，通过tf.shape函数得到输入的batch大小。
zero_state(
    batch_size,
    dtype
)

In [7]:
output = x
for i in range(5):
    output, state = stacked_lstm(output, state)
print(output.shape)

(?, 784)


将RNN在时间上展开，这里总共展开了5步，output为网络输出。

In [8]:
y_rnn = tf.layers.dense(output, 10,
                    kernel_initializer=tf.truncated_normal_initializer(stddev=0.1),
                    bias_initializer=tf.constant_initializer(0.1),
                    name='rnn_output_layer'
                    )

将RNN的输出经过全连接层，转化为对应的维度（10维，对应数字0-9）

In [9]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_rnn))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_rnn, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(2000):
        batch = mnist.train.next_batch(50)
        _,acc=sess.run([train_step,accuracy],feed_dict={x: batch[0], y_: batch[1]})
        if i % 10 == 0:            
            print('step %d, training accuracy %g' % (i, acc))
    ac=sess.run(accuracy,feed_dict={x: mnist.test.images, y_: mnist.test.labels})
    print('test accuracy %g' %ac)

step 0, training accuracy 0.1
step 10, training accuracy 0.46
step 20, training accuracy 0.62
step 30, training accuracy 0.56
step 40, training accuracy 0.74
step 50, training accuracy 0.76
step 60, training accuracy 0.82
step 70, training accuracy 0.84
step 80, training accuracy 0.8
step 90, training accuracy 0.86
step 100, training accuracy 0.9
step 110, training accuracy 0.84
step 120, training accuracy 0.84
step 130, training accuracy 0.82
step 140, training accuracy 0.86
step 150, training accuracy 0.94
step 160, training accuracy 0.9
step 170, training accuracy 0.88
step 180, training accuracy 0.88
step 190, training accuracy 0.84
step 200, training accuracy 0.88
step 210, training accuracy 0.84
step 220, training accuracy 0.92
step 230, training accuracy 0.9
step 240, training accuracy 0.8
step 250, training accuracy 0.94
step 260, training accuracy 0.96
step 270, training accuracy 0.88
step 280, training accuracy 0.96
step 290, training accuracy 0.92
step 300, training accuracy